<a href="https://colab.research.google.com/github/xhxuciedu/CS206/blob/master/krylov_subspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Krylov subspace methods**

Krylov subspace methods are a family of algorithms for solving $Ax=b$ that search for an approximate solution from a Krylov subspace. They are iterative, as opposed to direct, algorithms and usually require a fast matrix-vector product for $A$  (and possibly  $A^T$ ). The most famous Krylov subspace methods are Arnoldi, Lanczos, conjugate gradient, BiCGSTAB, and GMRES. We do not have time to cover them all. Here, we will focus on the GMRES method.



## **Initial thoughts on Krylov subspaces**

Let  $A$  be an invertible matrix and suppose we want to solve  $Ax=b$ , but we only know  $A$  via matrix-vector products, i.e., a function  $v\to Av$ . Let's consider the sequence of vectors  $b,Ab,…,A^{n−1}b,A^nb,…,$ 
 (not much else one can consider!). If we consider the first  $n+1$ , i.e.,  
 $$b,Ab,…,A^{n−1}b,A^nb$$ , then we are guaranteed for these vectors to be linear dependent ( $n+1$  vectors in  $n$-dimensional space must be linear dependent). Therefore, these exist coefficients  $α_0,…,α_n$  such that
$$α_0b+α_1Ab+⋯+α_nA^nb=0.$$
 
Let  $k$  be the smallest integer so that  $α_k≠0$ . Then, since  $A^{−1}$  exists:
$$A^{−1}b=−\frac{1}{α_k}(α_{k+1}b+⋯+α_nA^{n−k−1}b).$$
(Weak Cayley-Hamilton theorem). This shows that  $x=A^{−1}b$  can be computed by only matrix-vector products. This gives us an idea to search for "good" solutions from Krylov subspaces.

## The Krylov subspace
Let  $A$  be a matrix and  $c$  a vector. The  $r$th Krylov subspace, denoted by  $\mathcal{K}_r(A,c)$  is the vector spaced spanned by the vectors,
$$c,Ac,…,A^{r−1}c.$$
 
Usually, but not necessarily, the vector  $c$  is the right-hand side in a linear system  $Ax=b$  so that  $c=b$.



## Generalized minimum residual method (GMRES)
The GMRES is a Krylov subspace method that computes at the  $r$th step the best least squares solution  $x_r$  from the Krylov subspace  $\mathcal{K}_r(A,b)$. That is, the GMRES method successively solves the following least squares problems:

$$
\begin{array}
\textrm{Step 1} & \min_{x_1∈1}(A,b)‖b−Ax1‖2$,\\
\textrm{Step 2} & \min x2∈2(A,b)‖b−Ax2‖2 \\
\textrm{Step r} & \min_{xr∈r(A,b)} ‖b−Axr‖2.
\end{array}
 $$
These problems may seem to be awkward to solve at first because we do not know to find the least squares solution over Krylov subspaces. However, if the columns of  Qr  form an orthonormal basis for  r(A,b) , then we can equivalently write the GMRES method as
Step 1Step 2⋮Step rminc∈ℝ‖b−AQ1c‖2,x1=Q1cminc∈ℝ2‖b−AQ2c‖2,x2=Q2c⋮⋮minc∈ℝr‖b−AQrc‖2,xr=Qrc.
 
Therefore, for the  r th GMRES step we need to find the least squares solution to  AQrc=b . We do this by using the  QR  factorization of  AQr , i.e.,  AQr=Q̃ rR̃ r . Thus, we can rewrite GMRES again as the following steps:
Step 1Step 2⋮Step rSolve R̃ 1c=Q̃ T1b,Q̃ 1R̃ 1=AQ1,x1=Q1c,Solve R̃ 2c=Q̃ T2b,Q̃ 2R̃ 2=AQ2,x2=Q2c,⋮⋮Solve R̃ 1c=Q̃ T1b,Q̃ rR̃ r=AQr,xr=Qrc.
 
The GMRES method solves the above steps, but does so recursively so that the  r th step uses the fact that the  (r−1) th step has already been computed.

There are three questions remaining:

How to recursively construct an orthonormal basis for the Krylov subspaces?
How to recursively compute the  QR  factorization of  AQr ?
How many steps do we need before  ‖xr−A−1b‖2  is small?